In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
!pip install sentence-transformers
!pip install torchmetrics nltk
import nltk
from torchmetrics import F1
from sentence_transformers import SentenceTransformer

nltk.download('punkt')
rubert_sentence = SentenceTransformer('all-distilroberta-v1')

     |████████████████████████████████| 78 kB 1.7 MB/s eta 0:00:011
     |████████████████████████████████| 3.1 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 23.2 MB 166 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 3.0 MB/s eta 0:00:011
     |████████████████████████████████| 895 kB 8.4 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=b8cf9749def7b97742a7497a9bc0392824a46058ba7b184e40a6050978f69065
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
import numpy as np
augmented_chats = np.load("Dataset Large 200k/augmented_chats.npy")
augmented_targets = np.load("Dataset Large 200k/augmented_targets.npy")
augmented_chats.shape

(202464,)

In [7]:
# import re
# pattern = r"(<Builder>)?(<Architect>)?(\n)?"
# for i in range(len(augmented_chats)):
#   augmented_chats[i] = re.sub(pattern,"",augmented_chats[i])

In [8]:
chat_embeddings = []
for i in range(110000):
  chat_embeddings.append(rubert_sentence.encode(augmented_chats[i]))
  if(i%10000==0):
    print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [9]:
len(chat_embeddings)

110000

In [10]:
import numpy as np

all_targets = np.asarray(augmented_targets[:110000])
all_chat_embeddings = np.asarray(chat_embeddings)

train_targets = np.asarray(augmented_targets[:2500])
train_chat_embeddings = np.asarray(chat_embeddings[:2500])

#test_targets = np.asarray(augmented_targets[2500:])
# test_chat_embeddings = np.asarray(chat_embeddings[2500:])


In [11]:
len(all_targets)

110000

In [17]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from torch.nn.functional import one_hot

class TargetDataset(Dataset):
    def __init__(self, target_list, chat_list):
        self.target_list = target_list
        self.chat_list = chat_list

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx): 
        target = self.target_list[idx]
        target_tensor_target = torch.tensor(target, dtype=torch.long)
        chat_tensor = self.chat_list[idx]
        return chat_tensor, target_tensor_target
    
training_dataset = TargetDataset(train_targets, train_chat_embeddings)
test_dataset = TargetDataset(test_targets, test_chat_embeddings)

NameError: name 'train_targets' is not defined

In [13]:
train_set, val_set = torch.utils.data.random_split(training_dataset, [2300, 200])
train_dataloader = DataLoader(train_set, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

NameError: name 'training_dataset' is not defined

In [15]:
import torch
from torch import nn
from torch import optim
import numpy as np

device = torch.device('cuda')

class TargetDecoder(nn.Module):
    def __init__(self, features_dim=768): #768
        super(TargetDecoder, self).__init__()

        self.linear = nn.Sequential(nn.Linear(features_dim, 15680))

        self.cnn = nn.Sequential(
            nn.ConvTranspose3d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(), 
            nn.ConvTranspose3d(32, 7, kernel_size=3),
            # nn.Tanh()
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], 64, 5, 7, 7)
        x = self.cnn(x)
        return x

    

target_decoder = TargetDecoder()
target_decoder.load_state_dict(torch.load("110к_v2_target_decoder.pth"))
target_decoder = target_decoder.to(device)
loss_function = nn.CrossEntropyLoss()

In [16]:
from torchmetrics import Accuracy

accuracy = Accuracy()
EPOCHS = 100
optimizer = optim.Adam(target_decoder.parameters(), lr=1e-5)
count = 0
i = 0
for epoch in range(EPOCHS):
  count_per_epoch = 0
  target_decoder.train()
  train_loss = []
  train_acc = []
  for target_tensor_input, target_tensor_target in train_dataloader:
    i += 1
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    optimizer.zero_grad()
    predict = target_decoder(target_tensor_input)
    loss = loss_function(predict, target_tensor_target)
    _,pred_label = torch.max(predict, dim = 1)
    for k in range(len(pred_label)):
      if(np.all(pred_label[k].to("cpu").numpy() == target_tensor_target[k].to("cpu").numpy())):
        count+=1
        count_per_epoch += 1
    train_loss.append(loss.item())
    train_acc.append(accuracy(predict.to("cpu"),target_tensor_target.to("cpu")))
    loss.backward()
    optimizer.step()
  train_loss = np.array(train_loss).mean()
  train_acc = np.array(train_acc).mean()

  target_decoder.eval()
  val_loss = []
  val_acc = []
  for target_tensor_input, target_tensor_target in val_dataloader:
    i += 1
    target_tensor_input = target_tensor_input.float().to(device)
    predict = target_decoder(target_tensor_input)
    _,pred_label = torch.max(predict, dim = 1)
    for k in range(len(pred_label)):
      if(np.all(pred_label[k].to("cpu").numpy() == target_tensor_target[k].to("cpu").numpy())):
        count+=1
        count_per_epoch += 1
    loss = loss_function(predict.to(device), target_tensor_target.to(device))
    val_loss.append(loss.item())
    val_acc.append(accuracy(predict.to("cpu"), target_tensor_target.to("cpu")))
  
  val_loss = np.array(val_loss).mean()
  val_acc = np.array(val_acc).mean()
  print(f"epoch: {epoch} | loss: {train_loss} | val_loss: {val_loss}")
  print(f"train_acc: {train_acc} | val_acc: {val_acc} | count: {count_per_epoch}")
i = i/EPOCHS

NameError: name 'train_dataloader' is not defined

In [ ]:
count, i*8

(56269, 2504.0)

In [ ]:
f1 = F1(num_classes=7,mdmc_average="global", ignore_index=0)


In [ ]:
test_loss = []
test_acc = []
test_f1 = []
target_decoder.eval()
count = 0
i = 0
for target_tensor_input, target_tensor_target in test_dataloader:
  i+=1
  target_tensor_input = target_tensor_input.float().to(device)
  predict = target_decoder(target_tensor_input)
  loss = loss_function(predict.to(device), target_tensor_target.to(device))
  _,pred_label = torch.max(predict, dim = 1)
  if(np.all(pred_label.to("cpu").numpy() == target_tensor_target.to("cpu").numpy())):
    count+=1
  test_loss.append(loss.item())
  test_acc.append(accuracy(predict.to("cpu"), target_tensor_target.to("cpu")))
  test_f1.append(f1(predict.to("cpu"), target_tensor_target.to("cpu")))
test_loss = np.array(test_loss).mean()
test_acc = np.array(test_acc).mean()
test_f1 = np.array(test_f1).mean()
print(f"test_loss: {test_loss} | test_acc: {test_acc} | f1: {test_f1}")

test_loss: 0.0004132878329616842 | test_acc: 0.9999068379402161 | f1: 0.997547447681427


In [ ]:
count, i

(339, 345)

In [ ]:
# 0.032, 0.9942 - distil_roberta

## Production

In [27]:
all_training_dataset = TargetDataset(all_targets, all_chat_embeddings)
all_train_dataloader = DataLoader(all_training_dataset, batch_size=64, shuffle=True)

In [28]:
len(all_training_dataset)

110000

In [29]:
import torch
from torch import nn
from torch import optim
import numpy as np

device = torch.device('cuda')

class TargetDecoder(nn.Module):
    def __init__(self, features_dim=768): #768
        super(TargetDecoder, self).__init__()

        self.linear = nn.Sequential(nn.Linear(features_dim, 15680))

        self.cnn = nn.Sequential(
            nn.ConvTranspose3d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(), 
            nn.ConvTranspose3d(32, 7, kernel_size=3),
            # nn.Tanh()
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], 64, 5, 7, 7)
        x = self.cnn(x)
        return x

    

target_decoder = TargetDecoder()
target_decoder.load_state_dict(torch.load("110к_v2_target_decoder.pth"))
target_decoder = target_decoder.to(device)
loss_function = nn.CrossEntropyLoss()

In [30]:
from torchmetrics import Accuracy

optimizer = optim.Adam(target_decoder.parameters(), lr=1e-5)
accuracy = Accuracy()
EPOCHS = 5
count = 0
for epoch in range(EPOCHS):
  count_per_epoch = 0
  target_decoder.eval()
  train_loss = []
  train_acc = []
  for target_tensor_input, target_tensor_target in all_train_dataloader:
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    optimizer.zero_grad()
    predict = target_decoder(target_tensor_input)
    loss = loss_function(predict, target_tensor_target)
    _,pred_label = torch.max(predict, dim = 1)
    for k in range(len(pred_label)):
      if(np.all(pred_label[k].to("cpu").numpy() == target_tensor_target[k].to("cpu").numpy())):
        count+=1
        count_per_epoch += 1
    train_loss.append(loss.item())
    train_acc.append(accuracy(predict.to("cpu"),target_tensor_target.to("cpu")))
    loss.backward()
    optimizer.step()
  train_loss = np.array(train_loss).mean()
  train_acc = np.array(train_acc).mean()

  print(f"epoch: {epoch} | loss: {train_loss} | train_acc: {train_acc} | count: {count_per_epoch}")


epoch: 0 | loss: 0.0013127321869706753 | train_acc: 0.9995589852333069 | count: 95622
epoch: 1 | loss: 0.0012929003436391692 | train_acc: 0.9995642900466919 | count: 95854
epoch: 2 | loss: 0.0012760917907070103 | train_acc: 0.9995702505111694 | count: 96074
epoch: 3 | loss: 0.0012603598993509025 | train_acc: 0.9995751976966858 | count: 96247
epoch: 4 | loss: 0.001245835205016429 | train_acc: 0.9995806217193604 | count: 96492


In [31]:
torch.save(target_decoder.state_dict(), '110к_target_decoder2.pth')

In [32]:
from torchmetrics import Accuracy

optimizer = optim.Adam(target_decoder.parameters(), lr=1e-5)
accuracy = Accuracy()
EPOCHS = 15
count = 0
for epoch in range(EPOCHS):
  count_per_epoch = 0
  target_decoder.eval()
  train_loss = []
  train_acc = []
  for target_tensor_input, target_tensor_target in all_train_dataloader:
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    optimizer.zero_grad()
    predict = target_decoder(target_tensor_input)
    loss = loss_function(predict, target_tensor_target)
    _,pred_label = torch.max(predict, dim = 1)
    for k in range(len(pred_label)):
      if(np.all(pred_label[k].to("cpu").numpy() == target_tensor_target[k].to("cpu").numpy())):
        count+=1
        count_per_epoch += 1
    train_loss.append(loss.item())
    train_acc.append(accuracy(predict.to("cpu"),target_tensor_target.to("cpu")))
    loss.backward()
    optimizer.step()
  train_loss = np.array(train_loss).mean()
  train_acc = np.array(train_acc).mean()

  print(f"epoch: {epoch} | loss: {train_loss} | train_acc: {train_acc} | count: {count_per_epoch}")

epoch: 0 | loss: 0.00123210600631473 | train_acc: 0.9995846748352051 | count: 96666
epoch: 1 | loss: 0.0012204122335351777 | train_acc: 0.9995876550674438 | count: 96853
epoch: 2 | loss: 0.0012080146013689322 | train_acc: 0.9995930790901184 | count: 97048
epoch: 3 | loss: 0.001196498725084285 | train_acc: 0.9995960593223572 | count: 97161
epoch: 4 | loss: 0.001185687799148241 | train_acc: 0.9996000528335571 | count: 97267
epoch: 5 | loss: 0.0011759062583669056 | train_acc: 0.999603271484375 | count: 97429
epoch: 6 | loss: 0.001165295802137718 | train_acc: 0.9996068477630615 | count: 97540
epoch: 7 | loss: 0.001155899860266349 | train_acc: 0.9996095895767212 | count: 97715
epoch: 8 | loss: 0.0011470109618372454 | train_acc: 0.9996129870414734 | count: 97824
epoch: 9 | loss: 0.0011380694764519979 | train_acc: 0.9996153116226196 | count: 97942
epoch: 10 | loss: 0.001130011753974062 | train_acc: 0.999618649482727 | count: 98043
epoch: 11 | loss: 0.001121617855749379 | train_acc: 0.99962037

In [33]:
torch.save(target_decoder.state_dict(), '110к_target_decoder3.pth')

In [ ]:
from torchmetrics import Accuracy

optimizer = optim.Adam(target_decoder.parameters(), lr=1e-5)
accuracy = Accuracy()
EPOCHS = 45
count = 0
for epoch in range(EPOCHS):
  count_per_epoch = 0
  target_decoder.eval()
  train_loss = []
  train_acc = []
  for target_tensor_input, target_tensor_target in all_train_dataloader:
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    optimizer.zero_grad()
    predict = target_decoder(target_tensor_input)
    loss = loss_function(predict, target_tensor_target)
    _,pred_label = torch.max(predict, dim = 1)
    for k in range(len(pred_label)):
      if(np.all(pred_label[k].to("cpu").numpy() == target_tensor_target[k].to("cpu").numpy())):
        count+=1
        count_per_epoch += 1
    train_loss.append(loss.item())
    train_acc.append(accuracy(predict.to("cpu"),target_tensor_target.to("cpu")))
    loss.backward()
    optimizer.step()
  train_loss = np.array(train_loss).mean()
  train_acc = np.array(train_acc).mean()

  print(f"epoch: {epoch} | loss: {train_loss} | train_acc: {train_acc} | count: {count_per_epoch}")

epoch: 0 | loss: 0.001036684111421764 | train_acc: 0.9996472001075745 | count: 99137
epoch: 1 | loss: 0.001031485120685952 | train_acc: 0.9996493458747864 | count: 99219
epoch: 2 | loss: 0.0010262651942265228 | train_acc: 0.999651312828064 | count: 99321
epoch: 3 | loss: 0.0010208467381801554 | train_acc: 0.9996526837348938 | count: 99340
epoch: 4 | loss: 0.0010155332422031847 | train_acc: 0.9996544122695923 | count: 99397
epoch: 5 | loss: 0.0010109361538885569 | train_acc: 0.9996553063392639 | count: 99440
epoch: 6 | loss: 0.0010060920231777053 | train_acc: 0.9996576309204102 | count: 99546
epoch: 7 | loss: 0.001001237178684873 | train_acc: 0.9996596574783325 | count: 99606
epoch: 8 | loss: 0.0009966855959530239 | train_acc: 0.9996613264083862 | count: 99656
epoch: 9 | loss: 0.0009918146328353164 | train_acc: 0.9996626377105713 | count: 99728
epoch: 10 | loss: 0.0009874384594949015 | train_acc: 0.999663233757019 | count: 99761
epoch: 11 | loss: 0.0009829087594912822 | train_acc: 0.999

In [ ]:
torch.save(target_decoder.state_dict(), '110к_target_decoder4.pth')